In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from Geometry.WCD import WCD

In [2]:
wcte = WCD('wcte', kind='WCTE')

In [4]:
# Show numbering and orientation of all mPMT and bar-end target numberings in the WCTE detector
# simplify to show numbering for the front ones, but not back ones (several different views needed)

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
y_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
color_mpmt_c = 0x333333
color_missing_mpmt = 0xa83273
missing_mpmts = [45, 85, 91, 99]
color_target = 0xad4113

n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

n_fd_point_mpmt = 40
fd_indices_mpmt = []
for i in range(n_fd_point_mpmt):
    fd_indices_mpmt.append([i,(i+1)%n_fd_point_mpmt])
    
n_point_ft = 20
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])
    
# draw the extent of the mpmt baseplates (except for the bottom endcap (not visible for the full survey)
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    if i_mpmt < 21:
        continue
    
    p = mpmt.get_placement('design')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
    direction_y = np.cross(direction_z,direction_x)
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    y_vec = list(np.array(direction_y)*vec_length)
    origins.append(location)
    if i_mpmt not in missing_mpmts:
        z_vecs.append(z_vec)
        x_vecs.append(x_vec)
        y_vecs.append(y_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    color = color_mpmt
    if i_mpmt in missing_mpmts:
        color = color_missing_mpmt
        
    indices = indices_mpmt
    if mpmt.kind == 'FD':
        indices = fd_indices_mpmt
    plt_baseplate = k3d.lines(baseplate_points, indices, indices_type='segment', color=color)
    plot += plt_baseplate
    
    if i_mpmt not in missing_mpmts:
        if mpmt.kind != 'FD':
            feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
            plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough
        else:
            feedthrough1_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough1'),dtype=np.float32)
            plt_feedthrough1 = k3d.lines(feedthrough1_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough1
            feedthrough2_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough2'),dtype=np.float32)
            plt_feedthrough2 = k3d.lines(feedthrough2_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough2
    
    # skip the following for mPMTs not in the foreground:
    fl_mpmt = [29,33,21,25][3]
    top_selection = {29:[100,101,102,103],33:[103,104,105,94],21:[94,95,96,97],25:[97,98,99,100]}[fl_mpmt]
    if (i_mpmt > 85 or (i_mpmt - fl_mpmt + 16)%16 > 3) and i_mpmt not in top_selection:
        continue
    
    
    # k3d complains about the following not being float32!
    color = color_mpmt_c
    if i_mpmt in missing_mpmts:
        color = color_missing_mpmt
    plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color)
    plot += plt_text
    
    # draw the survey C holes
    if 1==1 and i_mpmt not in missing_mpmts:
        for c in ['c1','c2','c3','c4']:
            survey_points = np.array(mpmt.get_xy_points('design', feature='survey_'+c),dtype=np.float32)
            plt_survey = k3d.lines(survey_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_survey
            # remove the text if it's too cluttered
            plt_text = k3d.text(c, position=survey_points[0], reference_point='cc', size=1., label_box=False, color=color_mpmt_c)
            plot += plt_text

# draw the targets
for sm in wcte.sms:
    first_target = {'bottom':0, 'barrel':0, 'top':16}[sm.name]
    target_origins = []
    if 1==1 and sm.targets is not None:
        for i_target, target in enumerate(sm.targets):
            
            p = target.get_placement('design')
            location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
            target_origins.append(location)
            text_location = [location[j] - 10.*direction_z[j] for j in range(3)]
            plt_text = k3d.text(str(i_target+first_target), position=text_location, reference_point='cc', size=1., label_box=False, color=color_target)
            plot += plt_text


plot.display()

Output()

In [6]:
wcte.sms[0].name

'bottom'